In [ ]:
np.log(1e-10)


In [ ]:
import numpy as np

EPSILON = 1e-10

class LogisticRegression:
    
    def __init__(self):
        self.w = None  # (m,)
        self.b = None

    # X: (n, m)
    # return: (n,)
    def predict(self, X):
        if self.w is None:
            raise ValueError("not initizlied")
        z = X @ self.w + self.b
        z = np.clip(z, -500,500)
        return 1 / (1 + np.exp(-z))

    def predict_classes(self, X):
        y = self.predict(X)
        return (y > 0.5).astype(int)

    # X: (n, m)
    # y: (n,)
    def loss(self, X, y):
        n = len(X)
        y_pred = self.predict(X)
        # clip for numerical stability
        y_pred = np.clip(y_pred, EPSILON, 1-EPSILON)
        return np.sum(-(y * np.log(y_pred) + (1-y) * np.log(1-y_pred))) / n

    def gradient_descent(self, X, y, learning_rate):
        n = len(X)
        y_pred = self.predict(X)
        # (m,) = (m, n) @ (n,)
        dldw = X.T @ (y_pred - y) / n
        dldb = np.sum(y_pred - y) / n
        self.w -= dldw * learning_rate
        self.b -= dldb * learning_rate

    def fit(self, X, y, steps, learning_rate=0.01):
        n = len(X)
        m = len(X[0])
        self.w = np.random.randn(m)
        self.b = 0
        
        for i in range(steps):
            self.gradient_descent(X, y, learning_rate)
            if i % 50 == 0:
                loss = self.loss(X, y)
                print(f"Step {i}, loss: {loss}")

In [ ]:
X = np.array([[1,2,3], [3,1,4]])
y = np.array([0,1])
l = LogisticRegression()


In [ ]:
l.fit(X, y, steps=801)

In [ ]:
l.predict(X)

## Test

In [ ]:
  # Test on a simple dataset
  from sklearn.datasets import make_classification
  from sklearn.model_selection import train_test_split

  from sklearn.preprocessing import StandardScaler

  X, y = make_classification(n_samples=1000, n_features=10, n_informative=8,
                             n_redundant=2, n_classes=2, random_state=42)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Add normalization
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Train your model
  model = LogisticRegression()
  model.fit(X_train, y_train, steps=1000, learning_rate=0.01)

  # Evaluate
  y_pred_probs = model.predict(X_test)
  y_pred = (y_pred_probs >= 0.5).astype(int)

  accuracy = np.mean(y_pred == y_test)
  print(f"Test Accuracy: {accuracy:.3f}")

  # Compare with sklearn
  from sklearn.linear_model import LogisticRegression as SklearnLR
  sklearn_model = SklearnLR()
  sklearn_model.fit(X_train, y_train)
  sklearn_acc = sklearn_model.score(X_test, y_test)
  print(f"Sklearn Accuracy: {sklearn_acc:.3f}")

In [ ]:
  import numpy as np
  from sklearn.datasets import load_breast_cancer
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import StandardScaler

  # Use a real, clean dataset
  data = load_breast_cancer()
  X, y = data.data, data.target

  # Split
  X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.2, random_state=42
  )

  # Normalize
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Verify normalization worked
  print(f"X_train mean: {X_train.mean():.6f} (should be ≈ 0)")
  print(f"X_train std: {X_train.std():.6f} (should be ≈ 1)")

  # Train your model
  model = LogisticRegression()
  model.fit(X_train, y_train, steps=500, learning_rate=0.1)

  # Test
  y_pred = model.predict_classes(X_test)
  accuracy = np.mean(y_pred == y_test)
  print(f"\nYour Model Accuracy: {accuracy:.3f}")

  # Compare with sklearn
  from sklearn.linear_model import LogisticRegression as SklearnLR
  sklearn_model = SklearnLR(max_iter=500)
  sklearn_model.fit(X_train, y_train)
  sklearn_acc = sklearn_model.score(X_test, y_test)
  print(f"Sklearn Accuracy: {sklearn_acc:.3f}")


## Review

🟡 Minor Improvements (Not critical, but good to know)

  1. Weight Initialization

  Currently:
  ```
  self.w = np.random.randn(m)  # Random normal
  ```
  Better for logistic regression:
  ```
  self.w = np.zeros(m)  # Or small random: np.random.randn(m) * 0.01
  ```

  Why:
  - **Logistic regression typically starts at zeros or very small values**
  - Large random weights can cause sigmoid saturation early
  - But your current approach works fine for simple cases!

  2. Return Predicted Classes

  Add a method to get binary predictions:
  ```
  def predict_classes(self, X, threshold=0.5):
      """Return binary class predictions (0 or 1)"""
      probabilities = self.predict(X)
      return (probabilities >= threshold).astype(int)
  ```

  This is useful because:
  - predict() returns probabilities [0, 1]
  - Often we need binary classifications {0, 1}
  - Interviewers might ask: "How do I get actual class predictions?"

  3. Consider Gradient Clipping (Optional)

  For numerical stability in gradient_descent:
  ```
  dldw = np.clip(X.T @ (y_pred - y) / n, -1, 1)  # Prevent exploding gradients
  ```

  But this is optional - your current version is fine!

  4. Method Naming (Very minor)
```
  def gradient_descent(self, X, y, learning_rate):
  ```

  Could be more specific:
  ```
  def gradient_descent_step(self, X, y, learning_rate):
  ```

  Since it performs one step, not the full gradient descent loop. But this is nitpicking!
